In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import os
import csv
from tqdm import tqdm
import faiss
from nltk.translate.bleu_score import sentence_bleu

In [2]:
!pip install nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 1.3 MB/s eta 0:00:02
     -- ------------------------------------- 0.1/1.5 MB 1.1 MB/s eta 0:00:02
     ---- ----------------------------------- 0.2/1.5 MB 1.2 MB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.5 MB 1.0 MB/s eta 0:00:02
     -------- ------------------------------- 0.3/1.5 MB 1.3 MB/s eta 0:00:01
     -------------- ------------------------- 0.6/1.5 MB 2.0 MB/s eta 0:00:01
     ----------------- ---------------------- 0.7/1.5 MB 2.0 MB/s eta 0:00:01
     ------------------------------ --------- 1.1/1.5 MB 3.0 MB/s eta 0:00:01
     ---------------------------------------  1.5/1.5 MB 3.5 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 3.4 MB/s eta 0:00:00
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/28/08/9dcdaa5aac4634e4c23af26d92121f7ce445c630efa0d3037881

In [2]:
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase


def preprocess(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'
    
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text

In [3]:
#importing bert tokenizer and loading the trained question embedding extractor model
from transformers import AutoTokenizer, TFAutoModel
biobert_tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/BioRedditBERT-uncased")
question_extractor_model1=tf.keras.models.load_model('question_extractor_model_2_11')

In [4]:
from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
gpt2_tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("./tf_gpt2_model_2_100")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./tf_gpt2_model_2_100.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
#preparing the faiss search
qa=pd.read_pickle('./train_gpt_data.pkl')
question_bert = qa["Q_FFNN_embeds"].tolist()
answer_bert = qa["A_FFNN_embeds"].tolist()
question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')

answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])

question_index = faiss.IndexFlatIP(question_bert.shape[-1])
answer_index.add(answer_bert)
question_index.add(question_bert)

In [6]:
#defining function to prepare the data for gpt inference
#https://github.com/ash3n/DocProduct
def preparing_gpt_inference_data(question,question_embedding):
  topk=20
  scores,indices=answer_index.search(
                  question_embedding.astype('float32'), topk)
  q_sub=qa.iloc[indices.reshape(20)]
  
  line = '`QUESTION: %s `ANSWER: ' % (
                        question)
  encoded_len=len(gpt2_tokenizer.encode(line))
  for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')
    encoded_len=len(gpt2_tokenizer.encode(line))
    if encoded_len>=1024:
      break
  return gpt2_tokenizer.encode(line)[-1024:]

In [7]:
def give_answer(question,answer_len):
  preprocessed_question=preprocess(question)
  question_len=len(preprocessed_question.split(' '))
  truncated_question=preprocessed_question
  if question_len>500:
    truncated_question=' '.join(preprocessed_question.split(' ')[:500])
  encoded_question= biobert_tokenizer.encode(truncated_question)
  max_length=256
  padded_question=tf.keras.preprocessing.sequence.pad_sequences(
      [encoded_question], maxlen=max_length, padding='post')
  question_mask=[[1 if token!=0 else 0 for token in question] for question in padded_question]
  embeddings=question_extractor_model1({'question':np.array(padded_question),'question_mask':np.array(question_mask)})
  gpt_input=preparing_gpt_inference_data(truncated_question,embeddings.numpy())
  mask_start = len(gpt_input) - list(gpt_input[::-1]).index(4600) + 1
  input=gpt_input[:mask_start+1]
  if len(input)>(1024-answer_len):
   input=input[-(1024-answer_len):]
  gpt2_output=gpt2_tokenizer.decode(tf_gpt2_model.generate(input_ids=tf.constant([np.array(input)]),max_length=1024,temperature=0.7)[0])
  answer=gpt2_output.rindex('`ANSWER: ')
  return gpt2_output[answer+len('`ANSWER: '):]
  

In [8]:
#defining the final function to generate answer assuming default answer length to be 20
def final_func_1(question):
  answer_len=50
  return give_answer(question,answer_len)

In [11]:
#defining the final function to calculate the required metric answer assuming default answer length to be 20
def final_func_2(question,answer):
  answer_len=20
  generated_answer=give_answer(question,answer_len)
  reference = [answer.split(' ')]
  candidate = generated_answer.split(' ')
  score = sentence_bleu(reference, candidate)
  return score

In [16]:
#testing final_func1 on sample input
answer=final_func_1("can sugar be absorbed through the skin ")
print("answer:",answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


answer: yes sugar is absorbed through the skin and is absorbed through the skin through the skin and through the skin is not a problem for people with diabetes or other diseases that require a liver transplant or surgery to remove the liver it is a good idea to consult


In [ ]:
'question_extractor_model_2_11'
"./tf_gpt2_model_2_100"
./train_gpt_data.pkl